---
### Step 0: Load The Data

In [1]:
# Import libraries
import math
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
from keras.models import Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Lambda, Conv2D, Activation, Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
#img_data_set = []
#label_data_set = []
lines = []
with open('labeled_images.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for line in reader:
            lines.append(line)
print(len(lines))

4110


In [3]:
#labeled_pictures = pd.read_csv('labeled_images.csv')
shuffle(lines)
train_data, val_data = train_test_split(lines, test_size = 0.2)
print(len(train_data))
print(len(val_data))

3288
822


---
### Step 1: Define Data Generator

In [4]:
def data_generator (data_lines, batch_size = 40) :
    num_lines = len(data_lines)
    while True:
        shuffle(data_lines)
        for offset in range(0, num_lines, batch_size):
            batch_lines = data_lines[offset:offset + batch_size]
            images = []
            labels = []
            for batch_line in batch_lines:
                #normal image
                img = cv2.cvtColor(cv2.imread('./rec_images/'+str(batch_line[0])+'.jpeg'), cv2.COLOR_BGR2HSV)
                resized_image = cv2.resize(img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                #rotate left
                img_center = tuple(np.array(img.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(img_center, -15, 1.0)
                rot_img = cv2.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv2.INTER_LINEAR)
                resized_image = cv2.resize(rot_img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                #rotate right
                rot_mat = cv2.getRotationMatrix2D(img_center, 15, 1.0)
                rot_img = cv2.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv2.INTER_LINEAR)
                resized_image = cv2.resize(rot_img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                #flip image vertically
                flip_img = cv2.flip(img, flipCode=1)
                resized_image = cv2.resize(flip_img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                #flip image horizontally
                flip_img = cv2.flip(img, flipCode=0)
                resized_image = cv2.resize(flip_img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                #flip image both directions
                flip_img = cv2.flip(img, flipCode=-1)
                resized_image = cv2.resize(flip_img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
                
            X = np.array(images)
            y = to_categorical(np.array(labels), num_classes=5)
            yield shuffle(X, y)

---
### Step 2: Define Model

In [ ]:
model = Sequential()
#normalize data
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(300,400,3)))

model.add(Conv2D(24,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(36,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(48,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit_generator(generator = data_generator(train_data),
                    validation_data = data_generator(val_data),
                    epochs = 10,
                    steps_per_epoch  = math.ceil(len(train_data) / 128),
                    validation_steps = math.ceil(len(val_data)   / 128)    )
model.save('model.h5')
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
print('Model saved')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
26/26 [==============================] - 387s 15s/step - loss: 1.4932 - acc: 0.2354 - val_loss: 1.2803 - val_acc: 0.2720
Epoch 2/10
26/26 [==============================] - 385s 15s/step - loss: 0.9065 - acc: 0.6021 - val_loss: 0.7163 - val_acc: 0.7226
Epoch 3/10
26/26 [==============================] - 436s 17s/step - loss: 0.5940 - acc: 0.7720 - val_loss: 0.4994 - val_acc: 0.8321
Epoch 4/10
26/26 [==============================] - 452s 17s/step - loss: 0.4048 - acc: 0.8607 - val_loss: 0.1927 - val_acc: 0.9268
Epoch 5/10
26/26 [==============================] - 432s 17s/step - loss: 0.1874 - acc: 0.9409 - val_loss: 0.1463 - val_acc: 0.9512
Epoch 6/10
26/26 [==============================] - 421s 16s/step - loss: 0.1085 - acc: 0.9644 - val_loss: 0.1003 - val_acc: 0.9669
Epoch 7/10
26/26 [==============================] - 393s 15s/step - loss: 0.0870 - acc: 

In [12]:
test_image = cv2.cvtColor(cv2.imread('./rec_images/5822.jpeg'), cv2.COLOR_BGR2HSV)
resized_image = cv2.resize(test_image, (400, 300))
resized_image = np.array([resized_image])
pred_vects = model.predict(resized_image)
print("Predicted Probabilities: {}".format(pred_vects))
pred_labels = pred_vects.argmax(axis=-1)
print("Predicted Labels: {}".format(pred_labels))

Predicted Probabilities: [[2.5236413e-05 1.6024712e-03 9.9814630e-01 1.0115817e-15 2.2598584e-04]]
Predicted Labels: [2]
